In [ ]:
!git clone https://github.com/tayssirmoussa66/reaction_dataset.git

Cloning into 'reaction_dataset'...
remote: Enumerating objects: 41251, done.
remote: Counting objects: 100% (453/453), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 41251 (delta 441), reused 447 (delta 435), pack-reused 40798
Receiving objects: 100% (41251/41251), 135.55 MiB | 19.48 MiB/s, done.
Resolving deltas: 100% (41103/41103), done.
Checking out files: 100% (41200/41200), done.


In [ ]:
class GCN(torch.nn.Module):

    def __init__(self, feature_size, n_layers, embedding_size, n_heads):
        super().__init__()

        self.layers = torch.nn.ModuleList()
           
        # construct input layer
        self.layers.append(GraphConv(feature_size, embedding_size))
        # construct hidden layers
        for i in range(n_layers -1):
            self.layers.append(GraphConv(embedding_size, embedding_size))

        self.att_layer = GATConv(embedding_size, embedding_size,heads=n_heads, 
                                    )
        
        self.linear1 = Linear(embedding_size, feature_size)
        

        self.Linear2 = Linear(embedding_size*n_heads, embedding_size)
        self.Linear3 = Linear(embedding_size*2, embedding_size)

        self.out_Layer= MLP(in_channels=feature_size*2, hidden_channels=10,
          out_channels=5, num_layers=6)
        self.relu = torch.nn.ReLU()

    def forward(self, x, edge_weight, edge_index):
        #Local embeddings 
        for layer in self.layers:
            x = layer( x, edge_index, edge_weight)
            x = F.relu(x)
        
         #attention layer
        
        y = self.att_layer(x, edge_index)
        y = self.relu(self.Linear2(y))
        y = F.dropout(y, p=0.6, training=self.training)

        #Global embeddings
        
        concat_vector = torch.cat([x, y], 1)
        concat_vector=self.Linear3(concat_vector)


        x = self.relu(self.linear1 (concat_vector))

        out = self.relu(self.out_Layer(torch.cat([x[edge_index[0]], x[edge_index[1]]], dim=-1)))

        return out
       

In [ ]:
!pip install wandb -qqq
import wandb
wandb.login()

     |████████████████████████████████| 1.8 MB 12.9 MB/s 
     |████████████████████████████████| 157 kB 40.1 MB/s 
     |████████████████████████████████| 181 kB 50.1 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
     |████████████████████████████████| 157 kB 32.7 MB/s 
     |████████████████████████████████| 157 kB 57.8 MB/s 
     |████████████████████████████████| 157 kB 64.4 MB/s 
     |████████████████████████████████| 157 kB 44.6 MB/s 
     |████████████████████████████████| 156 kB 46.8 MB/s 


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
!pip install rdkit-pypi -qqq

     |████████████████████████████████| 36.8 MB 1.3 MB/s 


In [ ]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 7.9 MB 11.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 3.5 MB 12.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 2.4 MB 12.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 709 kB 13.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.d

In [ ]:
from rdkit import Chem
import torch
from torch.nn import Linear, BatchNorm1d, ModuleList, Softmax
import torch_geometric
from torch_geometric.data import Dataset, Data, Batch, download_url
import numpy as np
import os.path as osp
from torch_geometric.loader import DataLoader
from tqdm import tqdm
from torch_geometric.nn import GraphConv,GATConv, MLP
import torch.nn.functional as F
from torch.autograd import Variable
from sklearn.metrics import confusion_matrix, f1_score, \
    accuracy_score, precision_score, recall_score, roc_auc_score

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def save_checkpoint(state, filename="/content/sample_data/my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    idx= checkpoint["batch_idx"]

In [ ]:
class GCN(torch.nn.Module):

    def __init__(self, feature_size, n_layers, embedding_size, n_heads):
        super().__init__()

        self.layers = torch.nn.ModuleList()
           
        # construct input layer
        self.layers.append(GraphConv(feature_size, embedding_size))
        # construct hidden layers
        for i in range(n_layers -1):
            self.layers.append(GraphConv(embedding_size, embedding_size))

        self.att_layer = GATConv(embedding_size, embedding_size,heads=n_heads, 
                                    )
        self.linear1 = Linear(embedding_size, feature_size)
        

        self.Linear2 = Linear(embedding_size*n_heads, embedding_size)
        self.Linear3 = Linear(embedding_size*2, embedding_size)

        self.out_Layer= MLP(in_channels=feature_size*2, hidden_channels=28,
          out_channels=5, num_layers=3)
        

    def forward(self, x, edge_weight, edge_index):
        #Local embeddings 
        for layer in self.layers:
            x = layer( x, edge_index, edge_weight)
            x = F.relu(x)
        
         #attention layer
        
        y = self.att_layer(x, edge_index)
        y = F.elu(self.Linear2(y))
        y = F.dropout(y, p=0.6, training=self.training)

        #Global embeddings
        Global = F.log_softmax(y, dim=1)
        concat_vector = torch.cat([x, Global], 1)
        concat_vector=self.Linear3(concat_vector)


        x = self.linear1 (concat_vector)

        out = self.out_Layer(torch.cat([x[edge_index[0]], x[edge_index[1]]], dim=-1))

        return out
       

In [ ]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import MultiStepLR

In [ ]:
pip install class-resolver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Loading the model
print("Loading model...")

loss_fn= torch.nn.CrossEntropyLoss()
model = GCN(feature_size=7, n_layers=10, embedding_size=100, n_heads=5) 
model.to(device)
optimizer = AdamW(model.parameters(), lr = 0.01 )

scheduler = MultiStepLR(optimizer, milestones = [400, 450], gamma = 0.1, verbose = False)


Loading model...


In [ ]:
model.train()

GCN(
  (layers): ModuleList(
    (0): GraphConv(7, 100)
    (1): GraphConv(100, 100)
    (2): GraphConv(100, 100)
    (3): GraphConv(100, 100)
    (4): GraphConv(100, 100)
    (5): GraphConv(100, 100)
    (6): GraphConv(100, 100)
    (7): GraphConv(100, 100)
    (8): GraphConv(100, 100)
    (9): GraphConv(100, 100)
  )
  (att_layer): GATConv(100, 100, heads=5)
  (linear1): Linear(in_features=100, out_features=7, bias=True)
  (Linear2): Linear(in_features=500, out_features=100, bias=True)
  (Linear3): Linear(in_features=200, out_features=100, bias=True)
  (out_Layer): MLP(14, 28, 28, 5)
)

In [ ]:
print(f"Cuda available: {torch.cuda.is_available()}")

Cuda available: False


In [ ]:
def calculate_metrics(y_pred, y_true, epoch):
   
    print(f"F1 Score: {f1_score(y_true, y_pred, average='micro')}")
    print(f"Accuracy: {accuracy_score(y_true, y_pred)}")
    prec = precision_score(y_true, y_pred, average='micro')
    rec = recall_score(y_true, y_pred, average='micro')
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
  

In [ ]:
def train_one_epoch( model,path, optimizer, loss_fn):
    running_loss = 0.0
    step = 0
   
   
    for i in range(400):

        data = torch.load(osp.join(path,f'data_{i}.pt'))    
         # Use GPU
        data.to(device)
        # Reset gradients
        
        optimizer.zero_grad() 
        # Passing the node features and the connection info
        
        output = model(data.x.float(), 
                                data.edge_weight.float(),
                                
                                data.edge_index,   
                                ) 
            
        # Calculating the loss and gradients

        loss = loss_fn(torch.abs(output), data.y)
        loss.backward()  
        optimizer.step()  

        # Update tracking
        running_loss += loss.item()
        step += 1
           
    checkpoint = {"state_dict": model.state_dict(), "optimizer": optimizer.state_dict()}
    save_checkpoint(checkpoint)
    return running_loss/step


In [ ]:
def test(epoch, model, path, loss_fn):
    all_preds = []
    all_labels = []
    running_loss = 0.0
    step = 0

    for i in range(400,499):

        data = torch.load(osp.join(path,f'data_{i}.pt'))   
        data.to(device)
        output = model(data.x.float(), 
                                data.edge_weight.float(),
                                data.edge_index,   
                                ) 
      
        loss =  loss_fn(torch.abs(output), data.y.type(torch.float))
         # Update tracking
        running_loss += loss.item()
        step += 1

        all_preds.append(np.rint(torch.abs(output).cpu().detach().numpy()))

        all_labels.append(data.y.cpu().detach().numpy())
    
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    
    print(np.unique(all_preds, return_counts =True))
    print(np.unique(all_labels, return_counts =True))
 
    calculate_metrics(all_preds, all_labels, epoch)
    
    return running_loss/step

In [ ]:
for epoch in range (100):
   model.train()
   loss = train_one_epoch( model,'/content/reaction_dataset/Train_Data', optimizer,loss_fn)
   print(f"Epoch {epoch} | Train Loss {loss}")
   model.eval()
   if epoch % 5 == 0:
      loss = test(epoch, model,'/content/reaction_dataset/Train_Data',loss_fn)
      print(f"Epoch {epoch} | Test Loss {loss}")
      scheduler.step()

=> Saving checkpoint
Epoch 0 | Train Loss 2.007393551617245e-09
(array([ 0., 18.], dtype=float32), array([745308, 186327]))
(array([0., 1.]), array([745308, 186327]))
F1 Score: 0.8000000000000002
Accuracy: 0.8
Precision: 0.8
Recall: 0.8
Epoch 0 | Test Loss 0.0
=> Saving checkpoint
Epoch 1 | Train Loss 1.9132314372749745e-09
=> Saving checkpoint
Epoch 2 | Train Loss 1.6527316457097567e-09
=> Saving checkpoint
Epoch 3 | Train Loss 1.1194434458830128e-09
=> Saving checkpoint
Epoch 4 | Train Loss 1.2446396645457302e-09
=> Saving checkpoint
Epoch 5 | Train Loss 4.1284584128922846e-10
(array([ 0., 17., 18.], dtype=float32), array([745308,  63957, 122370]))
(array([0., 1.]), array([745308, 186327]))
F1 Score: 0.8000000000000002
Accuracy: 0.8
Precision: 0.8
Recall: 0.8
Epoch 5 | Test Loss 0.0
=> Saving checkpoint
Epoch 6 | Train Loss 6.348107713359251e-10
=> Saving checkpoint
Epoch 7 | Train Loss 8.791430650987401e-10
=> Saving checkpoint
Epoch 8 | Train Loss 1.4061197889496878e-09
=> Saving c